<a href="https://colab.research.google.com/github/mosesyhc/de300-wn2024-notes/blob/main/examples/ex-word-count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Google drive for a permanent venv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Retrieving Java, Spark, and `findspark` in Python

In [ ]:
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
# spark setup
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
# findspark helps locate the environment variables
import findspark
findspark.init()

In [ ]:
# import spark modules
from pyspark.sql import SparkSession

spark = (SparkSession.builder
                     .appName("Analyzing an unknown article.")
                     .getOrCreate())


In [ ]:
spark

In [ ]:
spark.sparkContext

In [ ]:
sc = spark.sparkContext
sqlContext = spark
spark.sparkContext.setLogLevel('ERROR')

In [ ]:
# determine file path
dir_path = r'/content/drive/MyDrive/DATA_ENG 300/'
file_path = dir_path + "sur.txt"

## Read

In [ ]:
# documentation of read
spark.read??

In [ ]:
# read file from spark
article = spark.read.text(file_path)  #

In [ ]:
article

In [ ]:
article.printSchema()

In [ ]:
article.show()

In [ ]:
# default show arguments
article.show(20, truncate=False)

In [ ]:
article

In [ ]:
# retrieving first rows of the article
article.printSchema()

In [ ]:
# we can work with a dataframe by selecting the content
article.select(article.value)
article.show(10, truncate=False)

In [ ]:
from pyspark.sql.functions import col

# all of the following returns the same df
article.select(article.value)
article.select(article['value'])
article.select(col('value'))
article.select('value')

## Split

In [ ]:
# splitting the lines
from pyspark.sql.functions import col, split

In [ ]:
lines = article.select(split(col('value'), " ").alias('line'))

In [ ]:
lines.show(10, truncate=False)

In [ ]:
article.select(split(col('value'), " ")).printSchema()

In [ ]:
article.select(split(col('value'), " ").alias("line")).printSchema()

In [ ]:
lines = article.select(split(article.value, " ").alias("line"))

In [ ]:
lines.printSchema()

In [ ]:
lines.show()

## Explode / Tokenize

In [ ]:
from pyspark.sql.functions import explode

In [ ]:
words = lines.select(explode(col("line")).alias("word"))

In [ ]:
words.show()

In [ ]:
words.printSchema()

## Clean

In [ ]:
from pyspark.sql.functions import lower
words_lower = words.select(lower(col("word")).alias("word_lower"))

In [ ]:
words_lower.show(3)

In [ ]:
from pyspark.sql.functions import regexp_extract

words_clean = words_lower.select(
    regexp_extract(col("word_lower"), r"(\W+)?([a-z]+)", 2).alias("word")
)

In [ ]:
words_clean.show(30)

In [ ]:
words_nonull = words_clean.where(col("word") != "")

words_nonull.show(100)

## Count / Group

In [ ]:
groups = words_nonull.groupBy(col("word"))

groups

In [ ]:
counts = words_nonull.groupBy(col("word")).count()

In [ ]:
counts

In [ ]:
counts.orderBy('count', ascending=False).show(15)

In [ ]:
counts.orderBy("count", ascending=False).show(10)

## All in one go!

In [ ]:
import pyspark.sql.functions as F

counts = (
    spark.read.text(file_path)
     .select(F.split(F.col('value'), ' ').alias('line'))
     .select(F.explode(F.col('line')).alias('word'))
     .select(F.lower(F.col('word')).alias('word'))
     .select(F.regexp_extract(F.col('word'), r"(\W+)?([a-z]+)", 2).alias('word'))
     .where(F.col('word') != "")
     .groupby('word')
     .count()
)

In [ ]:
counts.orderBy('count', ascending=False).show(100)